In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
# Data Parse and Cleanup
df = pd.read_csv("./doc.csv")
df['Year']= pd.to_datetime(df['Year'])
df.set_index('Year')

,Statement
Year,
1993-02-03,A meeting of the Federal Open Market Committee...
1993-03-23,A meeting of the Federal Open Market Committee...
1993-05-18,A meeting of the Federal Open Market Committee...
1993-07-07,A meeting of the Federal Open Market Committee...
1993-08-17,A meeting of the Federal Open Market Committee...
...,...
2020-07-29,"The Federal Reserve, the central bank of the U..."
2020-09-16,"The Federal Reserve, the central bank of the U..."
2020-11-05,"The Federal Reserve, the central bank of the U..."


In [3]:
analyzer =  SentimentIntensityAnalyzer()
out_score = df['Statement'].apply(sentiment_ps)

KeyboardInterrupt: 

In [15]:
positive = []
subjectivity = []
negative = []
polarity = []
for x in out_score:
    positive.append(x['Positive'])
    polarity.append(x['Polarity'])
    negative.append(x['Negative'])
    subjectivity.append(x['Subjectivity'])
df['polarity'] = polarity
df['positive'] = positive
df['negative'] = negative
df['subjectivity'] = subjectivity
df.to_pickle("sentiment.pkl")

In [1]:
# If you don't want to reapply sentiment, just load the pickle and start here
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

NameError: name 'pd' is not defined

In [4]:
# Get the stock info
price_changes = []
price_change_percentages = []
labels = []
window = 5
# workaround for rolling widow
for index, row in df.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = start_price - stock_closing_sum
    price_change_percent = (start_price - stock_closing_sum) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    if(price_change > 0):
        labels.append(1)
    else:
        labels.append(0)
df['price_change'] = price_changes
df['price_change_percent'] = price_change_percentages
df['label'] = labels

In [5]:
df

,Statement,polarity,positive,negative,subjectivity,Year,price_change,price_change_percent,label
Year,,,,,,,,,
1993-02-03,A meeting of the Federal Open Market Committee...,-0.134503,74,97,0.080546,1993-02-03,0.970001,0.002169,1
1993-03-23,A meeting of the Federal Open Market Committee...,-0.302857,61,114,0.080571,1993-03-23,-3.209991,-0.007153,0
1993-05-18,A meeting of the Federal Open Market Committee...,-0.259615,77,131,0.093990,1993-05-18,-8.529999,-0.019372,0
1993-07-07,A meeting of the Federal Open Market Committee...,-0.156627,35,48,0.038444,1993-07-07,-7.250000,-0.016372,0
1993-08-17,A meeting of the Federal Open Market Committee...,-0.224299,83,131,0.097717,1993-08-17,-6.639984,-0.014654,0
...,...,...,...,...,...,...,...,...,...
2020-07-29,"The Federal Reserve, the central bank of the U...",-0.219048,41,64,0.043263,2020-07-29,-69.330078,-0.021277,0
2020-09-16,"The Federal Reserve, the central bank of the U...",0.064935,82,72,0.062424,2020-09-16,148.570068,0.043884,1
2020-11-05,"The Federal Reserve, the central bank of the U...",-0.349593,40,83,0.049200,2020-11-05,-26.560059,-0.007566,0


In [6]:
#Start feature dataset
X = df
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df['label'])

In [7]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)


In [8]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [9]:
predictions = model.predict(x_test)

In [10]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75        52
           1       0.00      0.00      0.00        35

    accuracy                           0.60        87
   macro avg       0.30      0.50      0.37        87
weighted avg       0.36      0.60      0.45        87



/home/jyk2149/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [12]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)

0.5977011494252874
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        52
           1       0.00      0.00      0.00        35

    accuracy                           0.60        87
   macro avg       0.30      0.50      0.37        87
weighted avg       0.36      0.60      0.45        87



/home/jyk2149/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
statement = "The Federal Reserve is committed to using its full range of tools to support the U.S. economy in this challenging time, thereby promoting its maximum employment and price stability goalsThe COVID-19 pandemic is causing tremendous human and economic hardship across the United States and around the world. Following a moderation in the pace of the recovery, indicators of economic activity and employment have turned up recently, although the sectors most adversely affected by the pandemic remain weak. Inflation continues to run below 2 percent. Overall financial conditions remain accommodative, in part reflecting policy measures to support the economy and the flow of credit to U.S. households and businesses. The path of the economy will depend significantly on the course of the virus, including progress on vaccinations. The ongoing public health crisis continues to weigh on economic activity, employment, and inflation, and poses considerable risks to the economic outlook. The Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. With inflation running persistently below this longer-run goal, the Committee will aim to achieve inflation moderately above 2 percent for some time so that inflation averages 2 percent over time and longer‑term inflation expectations remain well anchored at 2 percent. The Committee expects to maintain an accommodative stance of monetary policy until these outcomes are achieved. The Committee decided to keep the target range for the federal funds rate at 0 to 1/4 percent and expects it will be appropriate to maintain this target range until labor market conditions have reached levels consistent with the Committee's assessments of maximum employment and inflation has risen to 2 percent and is on track to moderately exceed 2 percent for some time. In addition, the Federal Reserve will continue to increase its holdings of Treasury securities by at least $80 billion per month and of agency mortgage‑backed securities by at least $40 billion per month until substantial further progress has been made toward the Committee's maximum employment and price stability goals. These asset purchases help foster smooth market functioning and accommodative financial conditions, thereby supporting the flow of credit to households and businesses.In assessing the appropriate stance of monetary policy, the Committee will continue to monitor the implications of incoming information for the economic outlook. The Committee would be prepared to adjust the stance of monetary policy as appropriate if risks emerge that could impede the attainment of the Committee's goals. The Committee's assessments will take into account a wide range of information, including readings on public health, labor market conditions, inflation pressures and inflation expectations, and financial and international developments."

In [14]:
sent = sentiment_ps(statement)

In [39]:

vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')

In [52]:
X = vect.fit_transform(df.Statement)
corpus = gensim.matutils.Sparse2Corpus(X)
id_map = dict((v,k) for k, v in vect.vocabulary_.items())
ldana = gensim.models.LdaModel(corpus=corpus, id2word=id_map, passes=5)

In [53]:
for idx, topic in ldamodel.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.033*"achieving" + 0.031*"1995" + 0.031*"2019" + 0.028*"2014" + 0.027*"access" + 0.023*"affecting" + 0.023*"appear" + 0.022*"calculated" + 0.020*"abated" + 0.019*"asked"


Topic: 1 
Words: 0.021*"2007" + 0.021*"2005" + 0.018*"600" + 0.018*"2010" + 0.018*"2009" + 0.017*"2012" + 0.017*"abate" + 0.016*"2008" + 0.016*"2006" + 0.016*"2020"


Topic: 2 
Words: 0.049*"characterized" + 0.048*"come" + 0.045*"cases" + 0.044*"converge" + 0.043*"continuation" + 0.041*"consultations" + 0.041*"communicate" + 0.041*"consult" + 0.036*"calculated" + 0.032*"bids"


Topic: 3 
Words: 0.012*"bloom" + 0.012*"anticipation" + 0.011*"boehne" + 0.011*"auction" + 0.011*"bowman" + 0.011*"assembly" + 0.011*"bies" + 0.011*"assess" + 0.011*"bias" + 0.011*"assisting"


Topic: 4 
Words: 0.020*"construct" + 0.018*"combination" + 0.017*"consider" + 0.017*"companies" + 0.017*"component" + 0.017*"continuous" + 0.017*"constraints" + 0.017*"consequently" + 0.017*"conference" + 0.016*"contribution"




In [58]:
corpus_transformed = ldana[corpus]
data_dtmna = pd.DataFrame(X.toarray())
X.toarray()

array([[ 0,  0, 11, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  1,  0],
       ...,
       [ 0,  0,  0, ...,  1,  1,  1],
       [ 0,  0,  0, ...,  1,  1,  1],
       [ 1,  1,  0, ...,  1,  0,  2]])